<a href="https://colab.research.google.com/github/mgnarag/machine-learning/blob/main/Testing_and_metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls drive/My\ Drive
file_path = "/content/drive/My Drive/CS284/Project/"

In [ ]:
import time
import pandas as pd


import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


from PIL import Image, ImageOps
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from scipy.ndimage import zoom

import cv2

In [15]:
from tensorflow import keras
from PIL import Image, ImageOps
import numpy as np
import time

def crop(im,n_size):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)):
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            data.append(im1)
    return data


filename = str("GT_input.tiff")
test = Image.open(file_path+ "Testing nbs/" + filename)
test = test.convert('RGB')
w_dirty, h_dirty = test.size


architecture = ['unet_maxpooling_']
size = ['32_', '64_']
section = ['whole','portion']

models = []
for m in range(0,1):
  for s in range(0,2):
    for c in range(0,2):
      models.append(str(architecture[m]+size[s]+section[c]))

METRIC = []


for mod in range(0,4):
  if mod in [0, 1, 4, 5]:
    n_size = 32
    print(n_size)
  if mod in [2, 3, 6, 7]:
    n_size = 64
    print(n_size)
  start = time.time()
  model = keras.models.load_model(file_path + models[mod])
  print(model)
  xx = int(w_dirty/n_size)
  final=[]

  for portion in range(0,xx):
      im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
      w1, h1 = im1.size
      w = int(w1/n_size)
      h = int(h1/n_size)

      neverbeforeseen = np.array(crop(im1,n_size))
      #encoded_imgs = autoencoder.encoder(neverbeforeseen).numpy()
      decoded_imgs = model.predict(neverbeforeseen)

      col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
      for i in range(2,h):
          col = np.vstack((col,decoded_imgs[i]))

      #y = np.where(col > 0.5,1,0) #round the values
      y= col
      y = (y * 255).astype('uint8')
      if portion == 0:
          final = y
      if portion > 0:
          final = np.hstack((final,y))

  final = np.squeeze(final)
  reconstructed = Image.fromarray(final)
  reconstructed.save(file_path + "Testing nbs/"+str(models[mod])+"_cleaned.png")

  end = time.time()
  print("The time of execution of above program is :",
        (end-start), "seconds")



32
2/2 [==============================] - 0s 6ms/step
The time of execution of above program is : 6.92466139793396 seconds
32
2/2 [==============================] - 0s 7ms/step
The time of execution of above program is : 5.646198511123657 seconds
64
1/1 [==============================] - 0s 28ms/step
The time of execution of above program is : 5.6520750522613525 seconds
64
1/1 [==============================] - 0s 19ms/step
The time of execution of above program is : 5.169931888580322 seconds


In [17]:
import cv2
from skimage.metrics import structural_similarity as ssim
import numpy as np
import pandas as pd

def calculate_ssim(img1, img2):
    # Calculate SSIM for each channel
    ssim_r = ssim(img1[:, :, 0], img2[:, :, 0],win_size=65)
    ssim_g = ssim(img1[:, :, 1], img2[:, :, 1],win_size=65)
    ssim_b = ssim(img1[:, :, 2], img2[:, :, 2],win_size=65)

    # Take the average of SSIM values for all channels
    ssim_index = (ssim_r + ssim_g + ssim_b) / 3.0

    return ssim_index

def calculate_psnr(img1, img2):
    # Calculate PSNR for each channel
    psnr_r = cv2.PSNR(img1[:, :, 0], img2[:, :, 0])
    psnr_g = cv2.PSNR(img1[:, :, 1], img2[:, :, 1])
    psnr_b = cv2.PSNR(img1[:, :, 2], img2[:, :, 2])

    # Take the average of PSNR values for all channels
    psnr_value = (psnr_r + psnr_g + psnr_b) / 3.0

    return psnr_value


# Load your colored images
image1 = cv2.imread(file_path+ "Testing nbs/GT_output.tiff")

METRIC = []
for mod in range(0,4):
  image2 = cv2.imread(file_path + "Testing nbs/"+str(models[mod])+"_cleaned.png")



  # Calculate PSNR
  psnr = calculate_psnr(image1, image2)
  print(psnr)

  # Calculate SSIM
  #ssim_ = calculate_ssim(image1, image2)
  ssim_ = ssim(image1, image2, win_size=65,  multichannel=True)
  print(ssim_)

  METRIC.append([models[mod],psnr,ssim_])


METRIC = pd.DataFrame(METRIC)
headers = ['Model','PSNR','SSIM' ]
METRIC.columns = headers
METRIC.to_csv(file_path + 'Metrics.csv')
#print(METRIC)

27.401128535414937


<ipython-input-17-49a962d24f77>:44: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_ = ssim(image1, image2, win_size=1025,  multichannel=True)


0.8999763691321401
26.544276323700206
0.8849103921697586
27.008353023513326
0.8928025682569468
25.81418947863824
0.8643650188570767


In [ ]:
ssim_ = ssim(image1, image2, win_size=1025,  multichannel=True)

print(f'SSIM Index: {ssim_}')

<ipython-input-86-5d96e6a81854>:1: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_ = ssim(image1, image2, win_size=1025,  multichannel=True)


SSIM Index: 0.6717284780302376


In [18]:
print(models)

['unet_maxpooling_32_whole', 'unet_maxpooling_32_portion', 'unet_maxpooling_64_whole', 'unet_maxpooling_64_portion']
